In [1]:
import os
import sys
sys.path.insert(0, "..")

import torch

from my_datasets import *

In [6]:
r, n = 128, 64
chain_len = 15
sp = None
rules_dict = random_rules_with_chain(
    num_rules = r,
    num_vars = n,
    ante_prob = 0.5,
    conseq_prob = 0.5,
    state_prob = sp,
    chain_len = chain_len,
    return_dict = True
)

In [7]:
init_state = rules_dict["init_state"]
rules = rules_dict["rules"]

In [8]:
proof = prove_theorem(
    rules[None,...],
    init_state.view(1,n),
    init_state = init_state.view(1,n))

In [9]:
proof

{'rules': tensor([[[0, 0, 0,  ..., 1, 0, 0],
          [0, 1, 1,  ..., 0, 0, 0],
          [0, 1, 1,  ..., 0, 1, 1],
          ...,
          [1, 0, 1,  ..., 1, 1, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 1, 1]]]),
 'theorem': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'qed': tensor([1]),
 'states': tensor([[[1, 1, 1,  ..., 0, 1, 0],
          [1, 1, 1,  ..., 0, 1, 0],
          [1, 1, 1,  ..., 1, 1, 0],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]]]),
 'hits': tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [1, 0, 0,  ..., 0, 0, 0],
          [1, 0, 0,  ..., 0, 0, 0],
          [1, 0, 0,  ..., 0, 0, 0]]]),
 'chain_len': 